 # 01 Build the Visibility plot for Deep Drilling fields

- Author Sylvie Dagoret-Campagne
- Creation : 2025-06-03


## Ckeck on airmass.org : https://airmass.org/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, quantity_support
plt.style.use(astropy_mpl_style)
quantity_support()
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.cm as cmx
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

from matplotlib.dates import (AutoDateLocator, YearLocator, MonthLocator,
                              DayLocator, WeekdayLocator, HourLocator,
                              MinuteLocator, SecondLocator, MicrosecondLocator,
                              RRuleLocator, rrulewrapper, MONTHLY,
                              MO, TU, WE, TH, FR, SA, SU, DateFormatter,
                              AutoDateFormatter, ConciseDateFormatter)

%matplotlib inline
import pandas as pd

In [ ]:
import lsst.geom as geom
import lsst
from lsst.geom import Angle

In [ ]:
import astropy.units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord
from astropy.coordinates import Angle
from astropy.time import Time, TimezoneInfo,TimeDelta
from astropy.coordinates import Longitude,Latitude

In [ ]:
import datetime as dt
from datetime import datetime
import timezonefinder, pytz
from tzwhere import tzwhere
from calendar import monthrange
import calendar

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
plt.rcParams["axes.labelsize"]="large"
plt.rcParams["axes.linewidth"]=2.0
plt.rcParams["xtick.major.size"]=8
plt.rcParams["ytick.major.size"]=8
plt.rcParams["ytick.minor.size"]=5
plt.rcParams["xtick.labelsize"]="large"
plt.rcParams["ytick.labelsize"]="large"

plt.rcParams["figure.figsize"]=(12,8)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
#plt.rcParams['axes.facecolor'] = 'blue'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['lines.markeredgewidth'] = 0.3 # the line width around the marker symbol
plt.rcParams['lines.markersize'] = 5  # markersize, in points
plt.rcParams['grid.alpha'] = 0.75 # transparency, between 0.0 and 1.0
plt.rcParams['grid.linestyle'] = '-' # simple line
plt.rcParams['grid.linewidth'] = 0.4 # in points
plt.rcParams['font.size'] = 13

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x: Angle(x, lsst.geom.degrees).asRadians()
func_zendtoAirmass = lambda x: 1.0 / np.cos(func_degToRad(x))
# func_wrap = lambda x : Angle(x,lsst.geom.radians).wrap(180.*lsst.geom.degrees)

In [ ]:
def ra_to_mollweide(ra_rad):
    """
    Transforme une ascension droite (RA en radians) pour une projection Mollweide :
    - RA doit être d'abord dans [0, 2π]
    - Puis ramené dans [-π, +π] pour correspondre au domaine Mollweide
    - Puis inversé pour que RA croisse vers la gauche sur la carte

    Paramètre:
    - ra_rad : array ou scalaire de RA en radians

    Retourne:
    - RA transformé pour projection Mollweide

    Discussion avec ChatGPT verifiée le 28/04/2025
    """

    # 1) Assurer que RA est entre 0 et 2π
    ra_rad = np.remainder(ra_rad + 2 * np.pi, 2 * np.pi)

    # 2) Remettre RA entre -π et +π
    ra_rad[ra_rad > np.pi] -= 2 * np.pi

    # 3) Inverser RA pour que RA croisse vers la gauche
    return -ra_rad

In [ ]:
def CalculateBounds(theta,thetamin,thetamax):
    """
    Handle the boundaries of angles
    
    parameters:
        theta : the input angle in degree to test
        thetamin: lower bound angle
        thetamax: upper bound angle
    """
    
    theta_min_angle = Longitude(thetamin,unit=u.degree)
    theta_max_angle = Longitude(thetamax,unit=u.degree)
    theta_angle = Longitude(theta,unit=u.degree)
    
    #print("longitude angles",theta_angle,theta_min_angle,theta_max_angle)
    
    wrap = 0
    
    if theta_min_angle.degree < theta_max_angle.degree:
        wrap = 360 * u.deg
    else:
        wrap = 180 * u.deg
        
    theta_min_angle.wrap_angle = wrap
    theta_max_angle.wrap_angle = wrap
    theta_angle.wrap_angle = wrap
       
    #print("wrap",wrap,"theta=",theta_angle,"theta_min=",theta_min_angle,"theta_max",theta_max_angle)
    return theta_angle.is_within_bounds(theta_min_angle,theta_max_angle)
    

## Configuration

### Specify the observation date

In [ ]:
NYEAR=2025
NMONTH=7
NDAY=15

## Initialisation

### Location of sites

In [ ]:
list_of_observation_sites = EarthLocation.get_site_names()

In [ ]:
print(list_of_observation_sites)

#### Greenwitch Observatory

In [ ]:
latitude_greenwitch = 51.476852*u.degree 
# There is a little shift between the prime meridian and the Greenwitch observatory
#longitude_greenwitch = -0.000500*u.degree
longitude_greenwitch = -0.00*u.degree
altitude_greenwitch = 68.0*u.m
site_Greenwitch = EarthLocation(lat=latitude_greenwitch, lon=longitude_greenwitch, height=altitude_greenwitch)

In [ ]:
site_Greenwitch 

In [ ]:
tf = timezonefinder.TimezoneFinder()
timezone_greenwitch_str = tf.certain_timezone_at(lat=latitude_greenwitch/u.degree , lng=longitude_greenwitch/u.degree)
print(f"Time zone at Greenwitch Observatory: {timezone_greenwitch_str}")

In [ ]:
# Not working (want to know where is the observatory relative to zone border)
tz_data = tf.get_geometry(tz_name=timezone_greenwitch_str)[0][0]
all_long,all_lat = np.array(tz_data)
min_longitude_tzgreen = all_long.min()
max_longitude_tzgreen = all_long.max()

In [ ]:
print(f" Check longitude : {min_longitude_tzgreen} < {longitude_greenwitch} < {max_longitude_tzgreen}")

#### Rubin-LSST Observatory

In [ ]:
LSSTNAME='Cerro Pachon'
site_LSST = EarthLocation.of_site(LSSTNAME)
longitude_lsst = site_LSST.lon
latitude_lsst = site_LSST.lat
altitude_lsst = site_LSST.height
site_LSST

In [ ]:
cerro_pachon = EarthLocation(lat = latitude_lsst, lon = longitude_lsst, height = altitude_lsst)
cerro_pachon 

In [ ]:
tf = timezonefinder.TimezoneFinder()
timezone_lsst_str = tf.certain_timezone_at(lat=latitude_lsst/u.degree , lng=longitude_lsst/u.degree)
print(f"Time zone at Rubin LSST Observatory: {timezone_lsst_str}")

In [ ]:
# Not working (want to know where is the observatory relative to zone border)
tz_data = tf.get_geometry(tz_name=timezone_lsst_str)[0][0]
all_long,all_lat = np.array(tz_data)
min_longitude_tzlsst = all_long.min()
max_longitude_tzlsst = all_long.max()

In [ ]:
print(f" Check longitude : {min_longitude_tzlsst} < {longitude_lsst} < {max_longitude_tzlsst}")

### UTC offset

- Quelle est l'offset number en UTC du chili
ChatGPT a dit :
Le Chili se situe normalement dans le fuseau horaire UTC-4 (Chili Standard Time, CLT). Cependant, il utilise également l’heure d’été (Chili Summer Time, CLST), qui est UTC-3.

Détail :
- UTC-4 (CLT) : heure standard, utilisée en hiver (approximativement d’avril à septembre).
- UTC-3 (CLST) : heure d’été, utilisée en été (approximativement de septembre à avril).

Actuellement (20 mai 2025) :
Le Chili est en heure d’hiver (CLT), donc l’offset UTC est -4.

#### UTC offset using longitude

In [ ]:
longitude_offset = longitude_lsst - longitude_greenwitch
longitude_offset

In [ ]:
longitude_offset.hour

In [ ]:
#utcoffset_number = timezone.utcoffset(dt).total_seconds()/60./60.
utcoffset_number1 = longitude_offset.hour
print("The UTC offset in Chile is ",utcoffset_number1," hours")

In [ ]:
utcoffset_number2  = (longitude_lsst * 24.0 / 360.0/u.deg)
utcoffset_number2 

- Notice that this offset calculated above is in sidereal time not in solar time

#### UTC offset Using timezone

In [ ]:
if timezone_lsst_str is None:
    print("Could not determine my time zone")
else:
    # Display the current time in that time zone
        
    timezone_lsst = pytz.timezone(timezone_lsst_str)
    timezone0 = pytz.timezone('UTC')
    
    dt = datetime.utcnow()
    utc_offset =  timezone_lsst.utcoffset(dt)
    
    print("\t - The UTC Time now %s" % dt)
    print(f"\t - the UTC offset in zone {timezone_lsst_str} is {utc_offset}")

In [ ]:
# This is the offset that works with the sidereal time calculated after
utcoffset_number3 = timezone_lsst.utcoffset(dt).total_seconds()/60./60.
utcoffset_number3

### Try to correct internally to the time zone

In [ ]:
dt_corr = (longitude_offset.hour - np.floor(longitude_offset.hour))
dt_corr

In [ ]:
utcoffset_number4 = utcoffset_number3 - dt_corr

#### the TimezoneInfo object from astropy

- https://docs.astropy.org/en/stable/api/astropy.time.TimezoneInfo.html
- create the TimezoneInfo object required for sideral time

Have to choose among the four calculations :**utcoffset_number1, utcoffset_number2, utcoffset_number3, utcoffset_number4**

- Only utcoffset_number3 gives the correct sidereal time within 2 minutes

In [ ]:
utcoffset_number = utcoffset_number3
tz_utc_minus_xx_hours = TimezoneInfo(utc_offset = utcoffset_number*u.hour)

In [ ]:
utcoffset_number

### Night of observation and local time

- creation of an observation date chosen very close to the local midnight from the chosen NYEAR,NMONTH,NDAY
- Note we provide the **tz_utc_minus_xx_hours** which provide the UTC offset of the site

In [ ]:
# datetime at local midnight in the timezone of Chile by providing the right TimezoneInfo() object set at site Timezone
night_obs = datetime(NYEAR, NMONTH, NDAY, 0, 0)
night_obs_midnight = datetime(NYEAR, NMONTH, NDAY, 23, 59,59,tzinfo = tz_utc_minus_xx_hours)
night_obs_midnight

In [ ]:
night_obs_str = night_obs.strftime("%Y-%m-%d %H:%M:%S")
print(f"Night of Observation : {night_obs_str}")

In [ ]:
#check the string of the date
night_obs_midnight_str = night_obs_midnight.strftime("%Y-%m-%d %H:%M:%S")
night_obs_midnight_str
print(f"Midnight of Night of Observation : {night_obs_midnight_str}")

### Compute UTC and Sideral Time at local midnight

We check here that the offset in time between UTC and Chile time is 3 hours

##### TimezoneInfo object from astropy
https://docs.astropy.org/en/stable/api/astropy.time.TimezoneInfo.html

In [ ]:
tz_utc = TimezoneInfo() 
print("\t - Local time in Santiago in (GMT-3) in winter : ",night_obs_midnight)
print("\t - Time in UTC at the Local Time above         : ",night_obs_midnight.astimezone(tz_utc))

#### Sidereal time at LSST site 

- https://docs.astropy.org/en/stable/time/index.html
- we compute here the local sideral time at midnight local time
- we need to provide the utc offset  from the object TimezoneInfo required later to compute the local siteral time

#### Sidereal time and angle at local time midnight

- this calculation has been checked in Stellarium at Cerro Pachon for this observation date and time
- using utc_offset3 based on timezone info, I have only 2 minutes differences

In [ ]:
# the sidereal time is constructed from 1) local clock wall time, 2) the UTC timezone 3) and the location of observation site on earth 
t_lsst = Time(night_obs_midnight.astimezone(tz_utc), scale='utc',location = site_LSST)
#t_sidereal_lsst = t_lsst.sidereal_time('apparent')  
t_sidereal_lsst = t_lsst.sidereal_time('mean')  
print("Sideral Time at LSST midnight:",t_sidereal_lsst)

#### The Sideral angle is measured from the meridian ( westward in north hemisphere)

In [ ]:
print("Sidereal Time angle at LSST midnight:",Angle(t_sidereal_lsst).to(u.degree))
print("Sidereal Time angle at LSST midnight:",Angle(t_sidereal_lsst).degree)

## Define galactic plane

In [ ]:
# Galactic plane
gal_long = np.linspace(-180.0, 180, 360)
gal_lat = np.zeros((360))
coordinates_galactic_planes = SkyCoord(l=gal_long * u.degree, b=gal_lat * u.degree, frame="galactic")
gp_radec = coordinates_galactic_planes.transform_to("icrs")

# new method in radian with ra_to_mollweide
gp_ra_toplot = ra_to_mollweide(gp_radec.ra.radian)
gp_dec_toplot = gp_radec.dec.radian

## Define deep fields

In [ ]:
# Coordonnées des Deep Drilling Fields (RA en degrés, Dec en degrés)
ddf_names = ["XMM-LSS", "COSMOS", "ECDFS", "ELAIS-S1", "EDFS_a", "EDFS_b"]
ddf_ra_deg = np.array([35.57, 150.11, 52.98, 9.45, 58.90, 63.60])  # approximatif
ddf_dec_deg = np.array([-4.82, 2.23, -28.12, -44.02, -49.32, -47.60])  # approximatif


df = pd.DataFrame()
df["name"] = ddf_names 
df["ra"] = ddf_ra_deg
df["dec"] = ddf_dec_deg

# sorting
df = df.sort_values(by='ra',ascending = True)
df = df.reset_index(drop=True)

ddf_names = df["name"].values
ddf_ra_deg = df["ra"].values
ddf_dec_deg = df["dec"].values

# Conversion en radians et adaptation pour Mollweide projection
ddf_ra_rad = np.radians(ddf_ra_deg)
ddf_dec_rad = np.radians(ddf_dec_deg)
ddf_ra_rad = ra_to_mollweide(ddf_ra_rad)

In [ ]:
df

In [ ]:
NTargets = len(df)
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NTargets)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NTargets), alpha=1)

In [ ]:
# plot
# x arg in rad must be (-2pi,2pi), y arg in rad  must be in ( -pi,pi )
fig = plt.figure(figsize=(14, 8))
# ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111, projection="mollweide")

# galactic plane
# ax.scatter(ra_to_mollweide(gp_ra_toplot.radian), gp_radec.dec.radian, c="r", label="Galactic Plane", s=1)
# galactic plane
ax.scatter(gp_ra_toplot, gp_dec_toplot, c="r", label="Galactic Plane", s=1)

# ax.scatter(ra-np.pi,dec,marker='+',c=all_colors_g)
# im = ax.scatter(ra_g.values-np.pi,dec_g.values,marker='+',s=50,lw=3,c=dt,cmap=palette_spectral)
#im = ax.scatter(
#    ra_to_mollweide(ra_g.values), dec_g.values, marker="+", s=50, lw=2, c=dt, cmap=palette_spectral
#)

# Ajout des Deep Drilling Fields
for i in range(len(ddf_names)):
    ax.plot(ddf_ra_rad[i], ddf_dec_rad[i], "o", markersize=10, color=all_colors[i], markeredgecolor="black")
    ax.text(
        ddf_ra_rad[i] + 0.05,
        ddf_dec_rad[i] + 0.05,
        ddf_names[i],
        fontsize=12,
        color=all_colors[i],
        ha="left",
        va="bottom",
        weight="bold",
    )

ax.legend()

#cbar = plt.colorbar(im, orientation="horizontal", label="time (days)")
plt.suptitle("Rubin-LSST DDF", fontsize=20, fontweight="bold")

plt.show()



## Filter Obervable targets according RA and magnitude and culmination angle
$$
HA = LST - RA
$$

where $HA$ means Hour Angle, $LST$ means Local sidereal time and $RA$ means Right ascension
- $-6H<HA<+6H \longrightarrow  LST-6H <RA< LST+6H $ 
- Select the target which Right-Asccention is +/- 6 hours from the sideral time
- The culmination Hour angle at the meridian must not be less than -6H to + 6H

#### - Criteria on Margin Angle

- **Strange but I cannot have a larger margin : need to understand**

In [ ]:
MarginAngle = Angle(6.0,u.hour)
MarginAngle

$$
RA = LST - HA(margin\; Angle)
$$

In [ ]:
#calculate tdege range of RA target to be visible within the MarginAngle
#ra_min = (t_sidereal_lsst - MarginAngle).degree
#ra_max = (t_sidereal_lsst + MarginAngle).degree
ra_min = Longitude((t_sidereal_lsst - MarginAngle).degree,unit=u.deg)
ra_max = Longitude((t_sidereal_lsst + MarginAngle).degree,unit=u.deg)

In [ ]:
ra_min

In [ ]:
ra_max

In [ ]:
#ra_min_angle = Angle(ra_min,u.degree)
#ra_max_angle = Angle(ra_max,u.degree)
ra_min_angle = Longitude(ra_min,u.degree)
ra_max_angle = Longitude(ra_max,u.degree)
#ra_min_angle.wrap_angle = 180 * u.deg
#ra_max_angle.wrap_angle = 180 * u.deg

In [ ]:
ra_min_angle.degree

In [ ]:
ra_max_angle.degree

#### - Criteria on culmination angle

- The culmination angle is obtained when HA = 0, then RA = LST

In [ ]:
culmin_angle_min = 30

### Algo for the selection of the target wrt RA selection , culmination angle, 

In [ ]:
latitude = latitude_lsst

all_flag_select = []

# numeric collections
all_ra_angles_deg = []
all_dec_angles_deg = []
all_altmax_angle_deg = []
all_zenithmin_angle_deg = []
all_magV = []

# loop on each entries in the df table to calculate angles
for index, row in df.iterrows(): 
    target_name = row["name"]
    
    # compute numeric values for ra-dec
    ra_angle = Longitude(row['ra'],unit = u.deg)  
    dec_angle = Angle(row["dec"],unit = u.deg) 

    
    # compute the culmination angle from the zenith angle
    # North hemisphere
    if latitude.deg >= 0:
        if dec_angle.deg >= latitude.deg:
            zenith_angle =   dec_angle - latitude
        else:
            zenith_angle = latitude - dec_angle
    # south hemisphere
    else:
        if dec_angle.deg < latitude.deg :
            zenith_angle =   latitude - dec_angle 
        else:
            zenith_angle =   dec_angle - latitude 
        
    culmination_angle = Angle(90.0, unit = u.deg) - zenith_angle


            
             
    all_ra_angles_deg.append(ra_angle.degree)
    all_dec_angles_deg.append(dec_angle.degree)
    all_altmax_angle_deg.append(culmination_angle.degree) 
    all_zenithmin_angle_deg.append(zenith_angle.degree )

    # test on ra angle
    flag1 = CalculateBounds(ra_angle.degree,ra_min_angle.degree,ra_max_angle.degree)
    
    flag2 = False
    
    # test on culmination angle
    if culmination_angle.degree > culmin_angle_min :
        flag2 = True
        
    
    flag = flag1 & flag2 
    

    if flag:
        print(f"{index}, {target_name} , RA = {ra_angle.hour:.2f} , DEC = {dec_angle.degree:.2f} , RA-MIN-MAX = {ra_min_angle.deg:.1f}-{ra_max_angle.deg:.1f},zenith_angle = {zenith_angle.degree:.2f}  culmination angle = {culmination_angle.degree:.2f} , flag_select = {flag}")
   
    all_flag_select.append(flag)                

In [ ]:
all_flag_select

## Add calculated numerical values to the pandas table

In [ ]:
df["ra_deg"] = all_ra_angles_deg
df["dec_deg"] = all_dec_angles_deg 
df["alt_max"] = all_altmax_angle_deg
df["zenith_min"] = all_zenithmin_angle_deg

In [ ]:
df

## Apply all selection cuts on RA, culmination and magnitude

In [ ]:
df["select"] = all_flag_select
cut = df["select"] == True

t=df[cut]
NTargets=len(t)

In [ ]:
t=df

In [ ]:
t.sort_values(by="ra_deg",ascending=True, inplace=True)

### Must reindex the table

In [ ]:
t.reset_index(drop=True,inplace=True)

In [ ]:
t

In [ ]:
NTargets = len(t)
print(NTargets)

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NTargets)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NTargets), alpha=1)

## Compute Target location before computing their sky trajectory

In [ ]:
all_target_names = t["name"].values
selected_target_locations = [ SkyCoord(row.ra_deg,row.dec_deg, unit="deg") for key, row in t.iterrows() ]

In [ ]:
all_ra_astroquery=[]
all_dec_astroquery=[]
for target_loc in selected_target_locations:
    all_ra_astroquery.append(target_loc.ra)
    all_dec_astroquery.append(target_loc.dec)

# Astronomical in Observation frame for each selected target

## Initialisation 

In [ ]:
utcoffset_number

In [ ]:
night_obs_midnight_str

In [ ]:
# Greenwitch time at local midnight time
midnight_utc = Time(night_obs_midnight_str) - utcoffset_number*u.hour
delta_midnight = np.linspace(-12, 12, 1000)*u.hour
sideral_times = Angle(t_sidereal_lsst) + Angle(delta_midnight)

# utc times around local midnight
times_evening_to_morning = midnight_utc + delta_midnight
frame_evening_to_morning = AltAz(obstime=times_evening_to_morning, location=cerro_pachon)

In [ ]:
times_evening_to_morning_datetime = times_evening_to_morning.to_datetime()

## Sun frame

In [ ]:
from astropy.coordinates import get_sun
sunaltazs_evening_to_morning = get_sun(times_evening_to_morning).transform_to(frame_evening_to_morning)

## Moon frame

In [ ]:
# Old version in conda_py310
#from astropy.coordinates import get_moon
#moon_evening_to_morning = get_moon(times_evening_to_morning)
#moonaltazs_evening_to_morning = moon_evening_to_morning.transform_to(frame_evening_to_morning)

In [ ]:
from astropy import coordinates

### Get moon ephemerides
- Checked sky trajectory at https://airmass.org/

In [ ]:
moon_evening_to_morning = coordinates.get_body("moon",times_evening_to_morning,location=site_LSST)
moonaltazs_evening_to_morning = moon_evening_to_morning.transform_to(frame_evening_to_morning)

## Targets Frame

In [ ]:
all_target_altazs_evening_to_morning = [target_location.transform_to(frame_evening_to_morning) for target_location in  selected_target_locations ]

## Distance to the moon

In [ ]:
all_target_distancetomoon_evening_to_morning = [] 
all_target_tagnames = all_target_names
num_target=0
for idx,targetcoordinateseries in enumerate(all_target_altazs_evening_to_morning): 
    N_coord = len(targetcoordinateseries)
    idx_coord = 0
    all_distances_sep = []
    for idx_coord in range(N_coord):
        distance_sep=targetcoordinateseries[idx_coord].separation(moonaltazs_evening_to_morning[idx_coord])
        all_distances_sep.append(distance_sep.degree)
    all_distances_sep= np.array(all_distances_sep)
    all_distances_sep_min = all_distances_sep.min()
    all_target_distancetomoon_evening_to_morning.append(all_distances_sep_min) 
    all_target_tagnames[idx] += f", d_m = {all_distances_sep_min:.0f}°"
    num_target+=1

In [ ]:
all_target_tagnames

### Update target tag

In [ ]:
t["tag"] = all_target_tagnames

In [ ]:
t

## Sort target according increasing max culminating time

In [ ]:
NTargets = len(t)
all_timemax=np.zeros(NTargets)
all_altitudesmax=np.zeros(NTargets)
for idx in np.arange(NTargets):
    altitudes=all_target_altazs_evening_to_morning[idx].alt
    idx_max=np.where(altitudes==altitudes.max())[0][0]
    all_timemax[idx]=delta_midnight[idx_max].value
    all_altitudesmax[idx]=altitudes.max().degree

In [ ]:
all_altitudesmax

In [ ]:
t["tmax"] = all_timemax

In [ ]:
all_timemax

In [ ]:
rank = np.argsort(all_timemax)
rank

In [ ]:
rankinv = np.argsort(rank)
rankinv

In [ ]:
#t.index[rank]

In [ ]:
t["rank_tmax"] = rankinv # rank in time
t["rank_num"] = np.arange(NTargets) # rank in original table

In [ ]:
#t = t.sort_values(by = ['tmax'])

### Must reindex dataframe for loop on targets (in sorting order)

In [ ]:
#t.reset_index(inplace=True)

In [ ]:
t.sort_values(by= "ra_deg",  axis=0, ascending=True, inplace=False)

In [ ]:
t['rank_num'] =  [0, 1, 2, 3, 4, 5]
# Normaliser les valeurs entre 0 et 1
norm = mcolors.Normalize(vmin=t['rank_num'].min(), vmax=t['rank_num'].max())

# Choisir une colormap continue (par ex. "coolwarm" ou "RdBu_r")
colormap = cm.get_cmap("coolwarm")  # bleu → rouge

# Appliquer une couleur discrète par ligne
t['color'] = t['rank_num'].apply(lambda x: mcolors.to_hex(colormap(norm(x))))

print(t)

# Plot

In [ ]:
all_target_names = t["name"].values
all_target_names

In [ ]:
fig = plt.figure(figsize=(18,12),layout="constrained")


# First Axis : target sky trajectories
# ====================================
ax=fig.add_subplot(1,1,1)

# plot sun
ax.plot(delta_midnight, sunaltazs_evening_to_morning.alt, color='r', ls=":",label='Sun',lw=5)
#plot moon
ax.plot(delta_midnight, moonaltazs_evening_to_morning.alt, color=[0.75]*3, ls='--', label='Moon',lw=5)


# loop on selected targets in pandas dataframe t according rank order
for index in range(NTargets):
    
    # select the good row in pandas dataframe from time ordering
    row = (t[t["rank_tmax"] == index]).iloc[0]
    

    # get its tag
    label = str(index+1) + ") : " + row["tag"]
    
    # rank in all_target_altazs_evening_to_morning table
    rank_num = row["rank_num"]
    tmax = row["tmax"]
    altmax = row["alt_max"]
    
    
  
    # plot trajectory
    ax.plot(delta_midnight, all_target_altazs_evening_to_morning[rank_num].alt,label=label, lw=2.5,color=all_colors[index])
    #ax.plot(delta_midnight, all_target_altazs_evening_to_morning[rank_num].alt,label=label, lw=2.5,color=t["color"].values)

    # plot label
    if all_target_altazs_evening_to_morning[rank_num].alt[-1]>0:
        ax.text(delta_midnight[-1], all_target_altazs_evening_to_morning[rank_num].alt[-1], f'{index+1}',color=all_colors[index],fontsize=20)
    if all_target_altazs_evening_to_morning[rank_num].alt[0]>0:
        ax.text(delta_midnight[0], all_target_altazs_evening_to_morning[rank_num].alt[0], f'{index+1}',color=all_colors[index],fontsize=20)
    if altmax>0 :
        ax.text(tmax, altmax, f'{index+1}',color=all_colors[index],fontsize=20)
    


ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,sunaltazs_evening_to_morning.alt < -0*u.deg, color='0.5', zorder=0)
ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,sunaltazs_evening_to_morning.alt < -18*u.deg, color='k', zorder=0)

ax.legend(loc='upper right')
ax.set_xlim(-12*u.hour, 12*u.hour)
ax.set_xticks((np.arange(13)*2-12)*u.hour)
ax.set_ylim(0*u.deg, 90*u.deg)
ax.set_xlabel('Hours from Midnight local time')
ax.set_ylabel('Altitude [deg]')
ax.grid(None)
tick_spacing = 1
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
title = "Observations at AuxTel at night " + night_obs_str.split(" ")[0]
ax.set_title(title)
#for label in ax.get_xticklabels(which='major'):
#    label.set(rotation=30, horizontalalignment='right')


# second horizontal axis : Sideral time axis
# ==========================================
ax2 = ax.twiny()
ax2.plot([sideral_times[0].hour,sideral_times[-1].hour],[0,0])
ax2.set_xlabel("sideral time (hour)")
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
ax2.grid(None)

# third horizontal axis : UTC time axis
# =====================================

ax3 = ax.twiny()
# Move twinned axis ticks and label from top to bottom
ax3.xaxis.set_ticks_position("bottom")
ax3.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
offset = -0.12
ax3.spines["bottom"].set_position(("axes", offset))

ax3.set_frame_on(True)
ax3.patch.set_visible(False) # mandatory
for sp in ax3.spines.values():
    sp.set_visible(False)
ax3.spines["bottom"].set_visible(True)
#fmt = '%m/%d %H:%M'
fmt = '%H:%M'
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter(fmt))
ax3.xaxis.set_major_locator(mdates.HourLocator(interval=1))

curvetoremove, = ax3.plot_date(times_evening_to_morning_datetime, sunaltazs_evening_to_morning.alt, color='r', ls=":",lw=0)
curvetoremove.remove()
ax3.set_xlim(times_evening_to_morning_datetime[0], times_evening_to_morning_datetime[-1])



ax3.tick_params(which='major', width=1.00, length=5)
ax3.tick_params(which='minor', width=0.75, length=2.5)
ax3.grid(True)
#plt.gcf().autofmt_xdate()  # orient date labels at a slant
# slant for this axis
for label in ax3.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

ax3.set_xlabel("UTC Time")


figname="RubinLSST_StarAlt{:4d}_{:d}_{:d}.png".format(NYEAR,NMONTH,NDAY)

plt.tight_layout()
plt.savefig(figname)

plt.show()

# To plot in staralt

https://www.ing.iac.es//Astronomy/telescopes/wht/catformat.html

In [ ]:
t_staralt = t[["name","ra","dec","rank_tmax"]]
t_staralt["equinox"] = np.full(NTargets,"J2000")
t_staralt = t_staralt.sort_values(by = ['rank_tmax'])
t_staralt = t_staralt.drop(['rank_tmax'], axis=1)

In [ ]:
t_staralt

In [ ]:
t_staralt.to_csv("staralt.csv",index=False,header=False, sep = " ")

In [ ]:
! cat staralt.csv

# Spheric plot

### Looking toward local meridian, toward the north
- star rise from east 
- star set at west
- south pole is 30 degree above horizon

### Looking toward the south pole
- star rotate clockwise around the pole

### Dans l’hémisphère sud, les étoiles tournent autour du pôle Sud céleste dans le sens des aiguilles d’une montre.
Explication rapide :
- Dans l’hémisphère nord, les étoiles tournent autour du pôle Nord céleste dans le sens inverse des aiguilles d’une montre.
- Dans l’hémisphère sud, c’est l’inverse : autour du pôle Sud céleste, le mouvement apparent des étoiles est horaire, donc dans le sens des aiguilles d’une montre.
Si tu observes le ciel depuis l’hémisphère sud, par exemple en regardant vers le pôle Sud céleste (près de l’étoile Sigma Octantis), les étoiles sembleront tourner dans ce sens.

In [ ]:
def makePolarPlot(azimuthsInDegrees, zenithAngles, marker=".",ax=None,
                       title=None, color=None, objName=None,lw=0.5):
    if ax==None:
        _ = plt.figure(figsize=(20, 10))
        ax = plt.subplot(111, polar=True)
        
    ax.plot([a*np.pi/180 for a in azimuthsInDegrees], zenithAngles, marker, c=color, label=objName,lw=lw)
    if title:
        ax.set_title(title, va='bottom')
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlim(0, 90)
    return ax

In [ ]:
_ = plt.figure(figsize=(20, 10))

ax = plt.subplot(111, polar=True)


darkTimeIndex=[sunaltazs_evening_to_morning.alt < -18*u.deg][0]

# compute zenith and azimuth angles
moon_azimuthsInDegrees = Angle(moonaltazs_evening_to_morning.az).deg
moon_zenithAngles = Angle(moonaltazs_evening_to_morning.zen).deg

#ax=makePolarPlot(moon_azimuthsInDegrees, moon_zenithAngles,ax=ax, marker="--",color="y", title = None, lw=1)

moon_azimuthsInDegreesNight=moon_azimuthsInDegrees[darkTimeIndex]
moon_zenithAnglesNight=moon_zenithAngles[darkTimeIndex]
ax=makePolarPlot(moon_azimuthsInDegreesNight, moon_zenithAnglesNight,ax=ax ,marker="*-",title=None, color = "y",objName="Moon",lw=10)

    
# loop on selected targets in pandas dataframe t according rank order
for index in range(NTargets):
    
    # select the good row in pandas dataframe from time ordering
    row = (t[t["rank_tmax"] == index]).iloc[0]
    

    # get its tag
    label = str(index+1) + ") : " + row["tag"]
    
    # rank in all_target_altazs_evening_to_morning table
    rank_num = row["rank_num"]
    tmax = row["tmax"]
    altmax = row["alt_max"]
    
    
    
    
    
    # compute zenith and azimuth angles
    azimuthsInDegrees=Angle(all_target_altazs_evening_to_morning[rank_num].az).deg
    zenithAngles = Angle(all_target_altazs_evening_to_morning[rank_num].zen).deg
    
    ax=makePolarPlot(azimuthsInDegrees, zenithAngles,ax=ax, marker="-",title=None, color=all_colors[index], lw=0.5)
    
    azimuthsInDegreesNight=azimuthsInDegrees[darkTimeIndex]
    zenithAnglesNight=zenithAngles[darkTimeIndex]
    ax=makePolarPlot(azimuthsInDegreesNight, zenithAnglesNight,ax=ax ,marker="*-",title=None, color=all_colors[index],objName=label,lw=0.5)
    
    thex=zenithAnglesNight[0]*np.sin( (azimuthsInDegreesNight[0]) *np.pi/180.)
    they=zenithAnglesNight[0]*np.cos( (azimuthsInDegreesNight[0])*np.pi/180.)
   
    #plt.text(thex, they,f'{rank}',color=all_colors[rank],fontsize=20)
    
 
ax.legend(bbox_to_anchor=(1.08, 1), prop={'size': 15}, loc='upper left')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_rlim(0, 90)
ax.set_xticklabels(['N', '', 'E', '', 'S', '', 'W', ''])
title = "Observations at AuxTel at night " + night_obs_str.split(" ")[0]
ax.set_title(title)